# Object Oriented Programming

## Custom classes


We've defined a Player class and set up the default __init__ method to accept a data row as an argument. We made a deliberate choice to split up the logic of players and teams so our code is easy to read and maintain. We also made the convenient choice to initialize our Player instances using a data row. That's because all of the information is present in a row, and it will make it easier to create Player objects from the data set later on.

In [2]:
import pandas as pd
nba = pd.read_csv("nba_players_2013.csv")

header=['player', 'pos', 'age', 'team']

print(nba.head())
nba = nba.values.tolist()
print(type(nba))

print(header)
print(nba[:5])
print(type(nba))

class Player():
    # The special __init__ function runs whenever a class is instantiated
    # The init function can take arguments, but self is always the first one
    # Self is just a reference to the instance of the class
    # It is automatically passed in when you instantiate an instance of the class
    def __init__(self, data_row):
        self.player_name = data_row[0]
        self.position = data_row[1]
        self.age = data_row[2]
        self.team = data_row[3]

# Initialize a player using the first row of our data set
first_player = Player(nba[0])

# Implement the Team class iterating over all players and adding them as Player classes:
class Team():
    def __init__(self, team_name):
        self.team_name = team_name
        # Team roster initially empty
        self.roster = []
        # Find the players for the roster in the data set
        for row in nba:
            if row[3] == "San Antonio Spurs":
                self.roster.append(Player(row))

                
spurs = Team("San Antonio Spurs")
print(spurs)


          player pos  age                   team   g  gs    mp   fg   fga  \
0     Quincy Acy  SF   23                    TOT  63   0   847   66   141   
1   Steven Adams   C   20  Oklahoma City Thunder  81  20  1197   93   185   
2    Jeff Adrien  PF   27                    TOT  53  12   961  143   275   
3  Arron Afflalo  SG   28          Orlando Magic  73  73  2552  464  1011   
4  Alexis Ajinca   C   25   New Orleans Pelicans  56  30   951  136   249   

     fg.     ...      drb  trb  ast  stl  blk  tov   pf   pts     season  \
0  0.468     ...      144  216   28   23   26   30  122   171  2013-2014   
1  0.503     ...      190  332   43   40   57   71  203   265  2013-2014   
2  0.520     ...      204  306   38   24   36   39  108   362  2013-2014   
3  0.459     ...      230  262  248   35    3  146  136  1330  2013-2014   
4  0.546     ...      183  277   40   23   46   63  187   328  2013-2014   

   season_end  
0        2013  
1        2013  
2        2013  
3        2013  


The Player and Team classes we've defined serve as blueprints that we can use to create instances of these classes. **Classes** and the **instances** of those classes, which are collectively known as **objects**, are fundamental to object-oriented programming.

## Instance methods

If we want to compute the **average age of the players on a team**, we would write a method for the Team class that does this. However, because this number can be different for each team, we want to make sure the method acts individually on specific instances of the Team class. We call these methods **instance methods**.
 
For method declarations, the **first argument to the method is always** `self`, even though we don't explicitly pass in self when we call the method. self is a reference to the current object we're working with. It's useful when we want to access properties of that object within the method we're defining.

In [3]:
class Team():
    def __init__(self, team_name):
        self.team_name = team_name
        # Team roster initially empty
        self.roster = []
        # Find the players for the roster in the data set
        for row in nba:
            if row[3] == self.team_name:
                self.roster.append(Player(row))
    def num_players(self):
        count = 0
        for player in self.roster:
            count += 1
        return count

    # Implement the average_age() instance method
    def average_age(self):
        age_sum = 0
        for player in self.roster:
            age_sum += player.age
        return age_sum / self.num_players()
            
            
spurs = Team("San Antonio Spurs")
spurs_num_players = spurs.num_players()

spurs_avg_age = spurs.average_age()
print(spurs_avg_age)

28.428571428571427


## Class methods

In traditional object-oriented programming, everything (yes, everything) is an object. Integers are objects, and so are Booleans. While Python isn't quite this object-oriented, objects are nonetheless abundant in the Python language. For example, the math.floor function is really just a class method for the math class. Class methods act on an entire class, rather than a particular instance of one. We often use them as utility functions.

Notice also that we've begun writing a class method for you. The `@classmethod` line that appears above it tells the Python interpreter that the method is a class method. You'll need to follow this pattern whenever you declare class methods.

In [4]:
import math

class Team():
    def __init__(self, team_name):
        self.team_name = team_name
        self.roster = []
        for row in nba:
            if row[3] == self.team_name:
                self.roster.append(Player(row))
    
    def num_players(self):
        count = 0
        for player in self.roster:
            count += 1
        return count
   
    def average_age(self):
        return math.fsum([player.age for player in self.roster]) / self.num_players()
    
    @classmethod
    def older_team(self, team1, team2):
        if (team1.average_age() > team2.average_age()):
            return team1
        else:
            return team2



In [5]:
old_team = Team.older_team(Team('New York Knicks'),Team('Miami Heat'))
print(old_team.team_name)

Miami Heat


## Inheritance 

In object-oriented programming, the concept of inheritance enables us to organize classes in a tree-like hierarchy, where the parent class has some traits that it passes on to its descendants. When we define a class, we specify a parent class from which it inherits. Inheriting from a class means that the behavior of the parent also exists in the child, but that the child can still define its own additional behavior.

Consider a Player class with generic information about NBA players. This would be very useful because players have a lot of things in common. However, we may also want to add specific behavior for different positions. We can define classes like Center, Forward, or Point Guard, each with behavior that's specific to that position. These classes would each specify Player as its parent class. They would all be siblings -- each would inherit the same behaviors from the Player class, while also having special behaviors of their own.

In Python 3, every class is a subclass of a generic object class. While this happens automatically when we don't specify an ancestor, it's sometimes good practice to be explicit. For the remainder of this mission, we'll specify when a class has object as its parent while we code. This is a good programming practice -- if we get into the habit of specifying a class's ancestry, we won't forget to specify a parent when it's something other than object. It's simply a way to form good habits.

### Overloading inherited behaviour

When a class inherits from a parent class, it acquires all of the behavior of that parent class. There are times when we don't want all of that behavior, though, or want to modify it slightly for our custom class. We use a technique called overloading to accomplish this.

Overloading inherited behavior involves assigning new behavior to our custom class. To accomplish this, we just redefine the method on our new class.

We'll be altering our Player class to support comparisons that use these operators:

These methods already exist in the object class by default, and we've used these operators to compare integers, floating point numbers (decimals), and strings. The operators work because classes like string have implemented them specifically. It's a bit difficult to understand why the object class would need to have these methods, however. The best way to wrap your head around this is through an example.

Let's consider the addition operator (+). The object class already defines a method for addition. The sum() function is defined using this addition method, but the object class doesn't really know how to add integers or floating points specifically.

However, the integer and float classes define their own addition method (thus overloading the object's addition method), and the sum() function will add the values together properly. This architecture is very powerful, because even though sum() only had to be defined once, we can call it on a multitude of classes and it will result in proper behavior. This is an example of the power of inheritance and overloading.

In [6]:
class Player(object):
    # The special __init__ function runs whenever a class is instantiated
    # The init function can take arguments, but self is always the first one
    # Self is just a reference to the instance of the class
    # It is automatically passed in when you instantiate an instance of the class
    def __init__(self, data_row):
        self.player_name = data_row[0]
        self.position = data_row[1]
        self.age = int(data_row[2])
        self.team = data_row[3]
    def __lt__(self, other):
        return self.age < other.age
    
    def __gt__(self,other):
        return self.age > other.age
    
    def __le__(self,other):
        return self.age <= other.age
    
    def __ge__(self,other):
        return self.age >= other.age
    
    def __eq__(self,other):
        return self.age == other.age
    
    def __ne__(self,other):
        return self.age != other.age
    # Implement the rest of the comparison operators here
    

carmelo = Player(nba[17])
kobe = Player(nba[68])

result = (carmelo != kobe)
print(result)

True


In [7]:
# Compare average ages of teams

import math

class Team(object):
    def __init__(self, team_name):
        self.team_name = team_name
        self.roster = []
        for row in nba:
            if row[3] == self.team_name:
                self.roster.append(Player(row))
    def num_players(self):
        count = 0
        for player in self.roster:
            count += 1
        return count
    def average_age(self):
        return math.fsum([player.age for player in self.roster]) / self.num_players()
    # Define operators here
    
    def __lt__(self,other):
        return self.average_age()<other.average_age()
    
    def __gt__(self,other):
        return self.average_age()<other.average_age()

    def __le__(self,other):
        return self.average_age() <= other.average_age()

    def __ge__(self,other):
        return self.average_age() >= other.average_age()

    def __eq__(self,other):
        return self.average_age() == other.average_age()

    def __ne__(self,other):
        return self.average_age() != other.average_age()

    
utah = Team('Utah Jazz')
detroit = Team('Detroit Pistons')
    
if utah < detroit:
    older_team = detroit
else: 
    older_team = utah
    
print(older_team.team_name)

Utah Jazz


### Min Max

A lot of interesting information is readily available to us now that we've implemented the comparison operations. That's because Python uses these comparisons to implement many utility functions. Now we're able to use those functions to analyze data in a new setting. By overloading methods, we've given ourselves access to powerful functions without having to implement tedious logic.

In [8]:
import math

class Team(object):
    def __init__(self, team_name):
        self.team_name = team_name
        self.roster = []
        for row in nba:
            if row[3] == self.team_name:
                self.roster.append(Player(row))
    def num_players(self):
        count = 0
        for player in self.roster:
            count += 1
        return count
    def average_age(self):
        return math.fsum([player.age for player in self.roster]) / self.num_players()
    def __lt__(self, other):
        return self.average_age() < other.average_age()
    def __gt__(self, other):
        return self.average_age() > other.average_age()
    def __le__(self, other):
        return self.average_age() <= other.average_age()
    def __ge__(self, other):
        return self.average_age() >= other.average_age()
    def __eq__(self, other):
        return self.average_age() == other.average_age()
    def __ne__(self, other):
        return self.average_age() != other.average_age()

team_names = ["Boston Celtics", "Brooklyn Nets", "New York Knicks", "Philadelphia 76ers", "Toronto Raptors", 
         "Chicago Bulls", "Cleveland Cavaliers", "Detroit Pistons", "Indiana Pacers", "Milwaukee Bucks",
         "Atlanta Hawks", "Charlotte Hornets", "Miami Heat", "Orlando Magic", "Washington Wizards",
         "Dallas Mavericks", "Houston Rockets", "Memphis Grizzlies", "New Orleans Pelicans", "San Antonio Spurs",
         "Denver Nuggets", "Minnesota Timberwolves", "Oklahoma City Thunder", "Portland Trail Blazers", "Utah Jazz",
         "Golden State Warriors", "Los Angeles Clippers", "Los Angeles Lakers", "Phoenix Suns", "Sacramento Kings"]

# Alter this list comprehension
teams = list([Team(name) for name in team_names])

oldest_team = max(teams)
youngest_team = min(teams)
sorted_teams = sorted(teams)

print(oldest_team.team_name)
print(youngest_team.team_name)

Miami Heat
Philadelphia 76ers


# Exception handling



In [9]:
import pandas as pd
chopsticks = pd.read_csv("chopsticks.csv")

chopsticks = chopsticks.values.tolist()

# Define the Trial class here
print(chopsticks[0])


class Trial(object):
    def __init__(self, data_row):
        self.efficiency = float(data_row[0])
        self.individual = int(data_row[1])
        self.chopstick_length = int(data_row[2])
        
first_trial = Trial(chopsticks[0])
print(first_trial.efficiency)
print(first_trial.individual)
print(first_trial.chopstick_length)

[19.55, 1.0, 180.0]
19.55
1
180


In [10]:
# Define the Chopstick class here

class Chopstick(object):
    def __init__(self, length):
        self.length = int(length)

        

mini_chopstick = Chopstick(100)
print(mini_chopstick.length)

100


In [11]:
class Chopstick(object):
    def __init__(self, length):
        self.length = length
        # Start our trial list empty
        self.trials = []
        # Now, fill our list with relevant trials
        for row in chopsticks:
            if int(row[2]) == self.length:
                self.trials.append(row)

medium_chopstick = Chopstick(240) 
print(medium_chopstick.trials)

[[21.34, 1.0, 240.0], [29.94, 2.0, 240.0], [32.95, 3.0, 240.0], [29.4, 4.0, 240.0], [22.32, 5.0, 240.0], [28.36, 6.0, 240.0], [28.49, 7.0, 240.0], [22.24, 8.0, 240.0], [36.15, 9.0, 240.0], [30.62, 10.0, 240.0], [26.53, 11.0, 240.0], [27.95, 12.0, 240.0], [31.49, 13.0, 240.0], [30.24, 14.0, 240.0], [24.8, 15.0, 240.0], [26.43, 16.0, 240.0], [29.35, 17.0, 240.0], [21.15, 18.0, 240.0], [29.18, 19.0, 240.0], [21.6, 20.0, 240.0], [25.39, 21.0, 240.0], [22.26, 22.0, 240.0], [24.85, 23.0, 240.0], [24.56, 24.0, 240.0], [16.35, 25.0, 240.0], [22.96, 26.0, 240.0], [25.82, 27.0, 240.0], [19.46, 28.0, 240.0], [23.6, 29.0, 240.0], [33.1, 30.0, 240.0], [27.13, 31.0, 240.0]]


## Try Except
Typically when we write Python code, the interpreter will raise an exception (report an error) and then continue executing the rest of the code. We'll see the exception, but our program will keep running as if it never happened. This is undesireable, because our program probably relies on the previous statements to succeed.

We want to handle exceptions by observing when they occur and reacting to them accordingly instead. This way, every piece of code that executes is deliberate, and we have complete control over what our program does. In Python, we use a try-except block to handle exceptions. Consider this piece of code:



In [12]:
try:
    impossible_value = int("Not an integer")
except ValueError:
    print("Cannot convert string to integer")

Cannot convert string to integer


When the Python interpreter sees this code, it attempts to execute the try section of the statement. If the interpreter raises any exceptions within the try section (if we hit some sort of error), our code will attempt to catch it, or handle it gracefully with different code. In our example, the except statement is that different code. It will catch the exception and print out our message because we anticipated that a ValueError could occur, and built the error handling in.



In [13]:
try:
    f = open("data.txt", "r")
    s = f.readline()
    i = float(s)
except ValueError:
    print("Cannot convert data to floating point value")
except IOError:
    print("Could not read file")

Could not read file


With Python, we have the ability to catch any exception by writing an except: section without specifying a particular error. This is a sort of "catch-all" that works like an else: section. Using a catch-all for exceptions is usually bad practice, however. Trying to catch every exception without being specific is dangerous because then we can't execute exception-specific logic, and it means we may not understand our code as fully as we should.



In [14]:
class Trial(object):
    def __init__(self, datarow):
        try:
            self.efficiency = float(datarow[0])
        except ValueError:
            self.efficiency = -1
        try:
            self.individual = int(datarow[1])
        except ValueError:
            self.individual = -1
        try:
            self.chopstick_length = int(datarow[2])
        except ValueError:
            self.chopstick_length = -1
  
chopsticks.append(['Not a float', 'Individual', 'Length'])

bad_trial = Trial(chopsticks[-1])

In [15]:
print(bad_trial.efficiency)

-1


In [16]:
print(bad_trial.chopstick_length)

-1


In [17]:
chopsticks.pop()

class Chopstick(object):
    def __init__(self, length):
        self.length = length
        self.trials = []
        for row in chopsticks:
            if int(row[2]) == self.length:
                tta = Trial(row)
                if (tta.efficiency != -1) and (tta.individual != -1) and (tta.chopstick_length != -1):
                    self.trials.append(tta)
    def num_trials(self):
        return len(self.trials)
    def avg_efficiency(self):
        efficiency_sum = 0
        for trial in self.trials:
            efficiency_sum += trial.efficiency
        return efficiency_sum / self.num_trials()
print(chopsticks)
bad_chopstick = Chopstick(400)

[[19.55, 1.0, 180.0], [27.24, 2.0, 180.0], [28.76, 3.0, 180.0], [31.19, 4.0, 180.0], [21.91, 5.0, 180.0], [27.62, 6.0, 180.0], [29.46, 7.0, 180.0], [26.35, 8.0, 180.0], [26.69, 9.0, 180.0], [30.22, 10.0, 180.0], [27.81, 11.0, 180.0], [23.46, 12.0, 180.0], [23.64, 13.0, 180.0], [27.85, 14.0, 180.0], [20.62, 15.0, 180.0], [25.35, 16.0, 180.0], [28.0, 17.0, 180.0], [23.49, 18.0, 180.0], [27.77, 19.0, 180.0], [18.48, 20.0, 180.0], [23.01, 21.0, 180.0], [22.66, 22.0, 180.0], [23.24, 23.0, 180.0], [22.82, 24.0, 180.0], [17.94, 25.0, 180.0], [26.67, 26.0, 180.0], [28.98, 27.0, 180.0], [21.48, 28.0, 180.0], [14.47, 29.0, 180.0], [28.29, 30.0, 180.0], [27.97, 31.0, 180.0], [23.53, 1.0, 210.0], [26.39, 2.0, 210.0], [30.9, 3.0, 210.0], [26.05, 4.0, 210.0], [23.27, 5.0, 210.0], [29.17, 6.0, 210.0], [30.93, 7.0, 210.0], [17.55, 8.0, 210.0], [32.55, 9.0, 210.0], [28.87, 10.0, 210.0], [26.53, 11.0, 210.0], [25.26, 12.0, 210.0], [25.65, 13.0, 210.0], [29.39, 14.0, 210.0], [23.26, 15.0, 210.0], [24.77,

In [18]:
print(bad_chopstick.trials)

[]


When we try to find the average efficiency for a chopstick length that isn't in our data set, we end up dividing by zero in our avg_efficiency method. Fortunately, this throws a exception, and we can catch it.



In [19]:
class Trial(object):
    def __init__(self, datarow):
        try:
            self.efficiency = float(datarow[0])
            self.individual = int(datarow[1])
            self.chopstick_length = int(datarow[2])
        except:
            self.efficiency = -1
            self.individual = -1
            self.chopstick_length = -1

class Chopstick(object):
    def __init__(self, length):
        self.length = length
        self.trials = []
        for row in chopsticks:
            if int(row[2]) == self.length:
                trial = Trial(row)
                if trial.individual >= 0:
                    self.trials.append(trial)
    def num_trials(self):
        return len(self.trials)
    def avg_efficiency(self):
        efficiency_sum = 0
        for trial in self.trials:
            efficiency_sum += trial.efficiency
        try:
            return efficiency_sum / self.num_trials()
        except ZeroDivisionError:
            return -1


bad_average = Chopstick(100).avg_efficiency()
print(bad_average)

-1


In [32]:
# Checking for the best chopstick length

chopstick_lengths = [180, 195, 210, 225, 240, 255, 270, 285, 300, 315, 330]

#Use a list comprehension to convert the list of chopstick lengths into a list of Chopstick instances.
# Store the resulting list in chopstick_list.

chopstick_list = [Chopstick(length) for length in chopstick_lengths]


In [35]:
class Trial(object):
    def __init__(self, datarow):
        try:
            self.efficiency = float(datarow[0])
            self.individual = int(datarow[1])
            self.chopstick_length = int(datarow[2])
        except:
            self.efficiency = -1
            self.individual = -1
            self.chopstick_length = -1

class Chopstick(object):
    def __init__(self, length):
        self.length = length
        self.trials = []
        for row in chopsticks:
            if int(row[2]) == self.length:
                trial = Trial(row)
                if trial.individual >= 0:
                    self.trials.append(trial)
                    
                    
    def __lt__(self,other):
        return self.avg_efficiency() < other.avg_efficiency()
    
    def __gt__(self,other):
        return self.avg_efficiency() > other.avg_efficiency()
    
    def __le__(self,other):
        return self.avg_efficiency() <= other.avg_efficiency()
    
    def __ge__(self,other):
        return self.avg_efficiency() >= other.avg_efficiency()
    
    
    def __eq__(self,other):
        return self.avg_efficiency() == other.avg_efficiency()
    def __ne__(self,other):
        return self.avg_efficiency() != other.avg_efficiency()
    
       
    
    
    def num_trials(self):
        return len(self.trials)
    def avg_efficiency(self):
        efficiency_sum = 0
        for trial in self.trials:
            efficiency_sum += trial.efficiency
        try:
            return efficiency_sum / self.num_trials()
        except ZeroDivisionError:
            return -1.0
        
chopstick_lengths = [180, 195, 210, 225, 240, 255, 270, 285, 300, 315, 330]

chopstick_list = [Chopstick(length) for length in chopstick_lengths]

most_efficient = max(chopstick_list)
print(most_efficient)

## Lambda Functions